In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/marketing-strategy-personalised-offer/sample.csv
/kaggle/input/marketing-strategy-personalised-offer/train_data.csv
/kaggle/input/marketing-strategy-personalised-offer/test_data.csv


## Basic Data Exploration

In [2]:
df = pd.read_csv(r"../input/marketing-strategy-personalised-offer/train_data.csv")

In [3]:
df

,offer expiration,income_range,no_visited_Cold drinks,travelled_more_than_15mins_for_offer,Restaur_spend_less_than20,Marital Status,restaurant type,age,Prefer western over chinese,travelled_more_than_25mins_for_offer,...,restuarant_opposite_direction_house,has Children,visit restaurant with rating (avg),temperature,Restaur_spend_greater_than20,Travel Time,Climate,drop location,Prefer home food,Offer Accepted
0,2days,₹100000 or More,4~8,1,less1,Married partner,4 star restaurant,36,0,0,...,0,0,4,67,less1,22,Spring,Location B,0,No
1,2days,₹87500 - ₹99999,4~8,0,4~8,Married partner,Take-away restaurant,50plus,0,0,...,0,1,3,89,1~3,18,Summer,Location B,0,Yes
2,2days,₹87500 - ₹99999,less1,1,1~3,Single,Cold drinks,26,1,0,...,1,1,4,67,less1,7,Winter,Location A,1,Yes
3,10hours,₹37500 - ₹49999,less1,0,1~3,Single,Take-away restaurant,46,1,0,...,0,1,3,89,1~3,7,Summer,Location C,0,No
4,2days,₹100000 or More,never,1,1~3,Single,4 star restaurant,21,0,1,...,1,0,3,40,less1,7,Summer,Location C,0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12374,10hours,₹25000 - ₹37499,less1,1,gt8,Unmarried partner,Restaurant with pub,50plus,0,0,...,1,1,1,89,never,22,Summer,Location B,0,Yes
12375,2days,₹25000 - ₹37499,4~8,0,never,Single,Take-away restaurant,21,0,0,...,1,1,2,67,never,18,Summer,Location B,0,Yes
12376,10hours,₹87500 - ₹99999,4~8,1,gt8,Married partner,Cold drinks,31,0,1,...,1,0,2,89,gt8,18,Spring,Location B,1,No
12377,10hours,₹12500 - ₹24999,4~8,1,less1,Married partner,2 star restaurant,26,0,0,...,1,0,1,89,less1,7,Summer,Location C,0,Yes


In [4]:
df["offer expiration"].unique()

array(['2days', '10hours'], dtype=object)

In [5]:
df["no_visited_Cold drinks"].unique()

array(['4~8', 'less1', 'never', '1~3', 'gt8', nan], dtype=object)

In [6]:
df.isna().sum()

offer expiration                            0
income_range                                0
no_visited_Cold drinks                    198
travelled_more_than_15mins_for_offer        0
Restaur_spend_less_than20                 121
Marital Status                              0
restaurant type                             0
age                                         0
Prefer western over chinese                 0
travelled_more_than_25mins_for_offer        0
travelled_more_than_5mins_for_offer         0
no_visited_bars                            93
gender                                      0
car                                     12268
restuarant_same_direction_house             0
Cooks regularly                             0
Customer type                               0
Qualification                               0
is foodie                                   0
no_Take-aways                             144
Job/Job Industry                            0
restuarant_opposite_direction_hous

In [7]:
df["car"].unique()

array([nan, 'Mazda5', 'Car that is too old to install Onstar :D',
       'crossover', 'Scooter and motorcycle', 'do not drive'],
      dtype=object)

In [8]:
df["Restaur_spend_greater_than20"].unique()

array(['less1', '1~3', 'never', 'gt8', nan, '4~8'], dtype=object)

In [9]:
df["no_Take-aways"].unique()

array(['1~3', 'gt8', '4~8', 'less1', nan, 'never'], dtype=object)

In [10]:
df["income_range"].unique().shape

(9,)

## Some preliminary nan replacement?

In [11]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
# Six columns have nan values. We will have to consider various strategies to handle them.

In [12]:
#Before doing any kind of imputations, first let's copy the data
data = df.copy()

In [13]:
#For now, strategy is to impute with the most frequent value.
#First step is to find the most frequent value for the nan columns.
# First column in consideration is "no_visited_Cold drinks"
df["no_visited_Cold drinks"].value_counts()

less1    3304
1~3      3147
never    2903
4~8      1776
gt8      1051
Name: no_visited_Cold drinks, dtype: int64

In [14]:
data["no_visited_Cold drinks"].replace(np.nan,"less1", inplace = True)

In [15]:
#Next column is "Restaur_spend_less_than20"
df["Restaur_spend_less_than20"].value_counts()

1~3      5248
4~8      3485
less1    2052
gt8      1265
never     208
Name: Restaur_spend_less_than20, dtype: int64

In [16]:
data["Restaur_spend_less_than20"].replace(np.nan,"1~3", inplace = True)

In [17]:
#Next column is "no_visited_bars"
df["no_visited_bars"].value_counts()

never    5064
less1    3397
1~3      2430
4~8      1047
gt8       348
Name: no_visited_bars, dtype: int64

In [18]:
data["no_visited_bars"].replace(np.nan,"never", inplace = True)

In [19]:
#Next column is car
df['car'].value_counts()

Car that is too old to install Onstar :D    24
Scooter and motorcycle                      23
do not drive                                22
Mazda5                                      21
crossover                                   21
Name: car, dtype: int64

In [20]:
data["car"].replace(np.nan,"Car that is too old to install Onstar :D", inplace = True)

In [21]:
df["no_Take-aways"].value_counts()

1~3      4650
4~8      4115
less1    1762
gt8      1568
never     140
Name: no_Take-aways, dtype: int64

In [22]:
data["no_Take-aways"].replace(np.nan,"1~3",inplace = True)

In [23]:
df["Restaur_spend_greater_than20"].value_counts()

less1    5992
1~3      3216
never    2033
4~8       711
gt8       267
Name: Restaur_spend_greater_than20, dtype: int64

In [24]:
data["Restaur_spend_greater_than20"].replace(np.nan,"less1",inplace = True)

In [25]:
#Now that all columns have been taken care, let's perform a sanity check
data.isna().sum()

offer expiration                        0
income_range                            0
no_visited_Cold drinks                  0
travelled_more_than_15mins_for_offer    0
Restaur_spend_less_than20               0
Marital Status                          0
restaurant type                         0
age                                     0
Prefer western over chinese             0
travelled_more_than_25mins_for_offer    0
travelled_more_than_5mins_for_offer     0
no_visited_bars                         0
gender                                  0
car                                     0
restuarant_same_direction_house         0
Cooks regularly                         0
Customer type                           0
Qualification                           0
is foodie                               0
no_Take-aways                           0
Job/Job Industry                        0
restuarant_opposite_direction_house     0
has Children                            0
visit restaurant with rating (avg)

In [26]:
df.isna().sum()

offer expiration                            0
income_range                                0
no_visited_Cold drinks                    198
travelled_more_than_15mins_for_offer        0
Restaur_spend_less_than20                 121
Marital Status                              0
restaurant type                             0
age                                         0
Prefer western over chinese                 0
travelled_more_than_25mins_for_offer        0
travelled_more_than_5mins_for_offer         0
no_visited_bars                            93
gender                                      0
car                                     12268
restuarant_same_direction_house             0
Cooks regularly                             0
Customer type                               0
Qualification                               0
is foodie                                   0
no_Take-aways                             144
Job/Job Industry                            0
restuarant_opposite_direction_hous

## Converting Categorical to Numerical Values

In [27]:
data["Offer Accepted"].replace("No",0, inplace = True)
data["Offer Accepted"].replace("Yes",1,inplace = True)
data["Offer Accepted"] = data["Offer Accepted"].astype(np.uint8)

In [28]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
data["temperature"] = ss.fit_transform(data["temperature"].values.reshape(-1,1))

In [29]:
from sklearn.preprocessing import MinMaxScaler
ss2 = StandardScaler()
data["Travel Time"] = ss2.fit_transform(data["Travel Time"].values.reshape(-1,1))

In [30]:
ss3 = StandardScaler()
data["visit restaurant with rating (avg)"] = ss3.fit_transform(data["visit restaurant with rating (avg)"].values.reshape(-1,1))

In [31]:
#Now indeed we have cleaned all the columns, next step is to get numerical values for categorical
data_numerical = pd.get_dummies(data)

In [32]:
data_numerical.columns

Index(['travelled_more_than_15mins_for_offer', 'Prefer western over chinese',
       'travelled_more_than_25mins_for_offer',
       'travelled_more_than_5mins_for_offer',
       'restuarant_same_direction_house', 'Cooks regularly', 'is foodie',
       'restuarant_opposite_direction_house', 'has Children',
       'visit restaurant with rating (avg)',
       ...
       'Restaur_spend_greater_than20_4~8', 'Restaur_spend_greater_than20_gt8',
       'Restaur_spend_greater_than20_less1',
       'Restaur_spend_greater_than20_never', 'Climate_Spring',
       'Climate_Summer', 'Climate_Winter', 'drop location_Location A',
       'drop location_Location B', 'drop location_Location C'],
      dtype='object', length=116)

## Basic Model

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.preprocessing import StandardScaler

In [34]:
X = data_numerical.drop("Offer Accepted", axis = 1)
Y = data_numerical["Offer Accepted"]
X_train,X_test, Y_train,Y_test = train_test_split(X,Y,random_state = 42,test_size = 0.2)

In [35]:
X_train

,travelled_more_than_15mins_for_offer,Prefer western over chinese,travelled_more_than_25mins_for_offer,travelled_more_than_5mins_for_offer,restuarant_same_direction_house,Cooks regularly,is foodie,restuarant_opposite_direction_house,has Children,visit restaurant with rating (avg),...,Restaur_spend_greater_than20_4~8,Restaur_spend_greater_than20_gt8,Restaur_spend_greater_than20_less1,Restaur_spend_greater_than20_never,Climate_Spring,Climate_Summer,Climate_Winter,drop location_Location A,drop location_Location B,drop location_Location C
1691,1,0,0,1,1,0,1,0,1,-0.694854,...,0,0,1,0,0,1,0,0,1,0
8979,0,1,0,1,0,1,0,1,0,1.415450,...,0,0,1,0,1,0,0,0,1,0
123,1,0,0,1,0,1,1,1,1,0.712015,...,0,0,0,0,1,0,0,1,0,0
7038,0,1,0,1,1,0,0,0,1,1.415450,...,0,0,0,0,0,1,0,0,0,1
1952,1,0,0,1,1,0,1,0,0,0.008581,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11964,0,1,0,1,1,0,0,0,0,0.008581,...,0,0,0,0,0,1,0,0,1,0
5191,1,1,1,1,0,0,0,1,1,0.712015,...,0,0,1,0,1,0,0,0,1,0
5390,1,1,0,1,0,0,0,1,1,-1.398289,...,0,0,1,0,0,1,0,1,0,0
860,0,1,0,1,0,1,1,1,0,-1.398289,...,0,0,0,1,0,1,0,1,0,0


In [36]:
#Model 1
"""
lr = LogisticRegression(random_state = 42, max_iter = 40000)
lr.fit(X_train,Y_train)
"""

'\nlr = LogisticRegression(random_state = 42, max_iter = 40000)\nlr.fit(X_train,Y_train)\n'

In [37]:
"""
Y_preds_train = lr.predict(X_train)
Y_preds = lr.predict(X_test)
"""

'\nY_preds_train = lr.predict(X_train)\nY_preds = lr.predict(X_test)\n'

In [38]:

from sklearn.metrics import classification_report
"""
clr = classification_report(Y_test,Y_preds)
"""

'\nclr = classification_report(Y_test,Y_preds)\n'

In [39]:
"""
print(clr)
"""

'\nprint(clr)\n'

In [40]:
# Model 2
"""
svc = SVC(random_state = 42, gamma = 1e-1,C = 10)
svc.fit(X_train,Y_train)
Y_preds_svc = svc.predict(X_test)
clr_svc = classification_report(Y_test,Y_preds_svc)
print(clr_svc)
"""

'\nsvc = SVC(random_state = 42, gamma = 1e-1,C = 10)\nsvc.fit(X_train,Y_train)\nY_preds_svc = svc.predict(X_test)\nclr_svc = classification_report(Y_test,Y_preds_svc)\nprint(clr_svc)\n'

In [41]:
"""
svc = SVC(random_state = 42, gamma = 1e-2,C = 100)
svc.fit(X_train,Y_train)
Y_preds_svc = svc.predict(X_test)
clr_svc = classification_report(Y_test,Y_preds_svc)
print(clr_svc)
"""

'\nsvc = SVC(random_state = 42, gamma = 1e-2,C = 100)\nsvc.fit(X_train,Y_train)\nY_preds_svc = svc.predict(X_test)\nclr_svc = classification_report(Y_test,Y_preds_svc)\nprint(clr_svc)\n'

In [42]:
"""
Y_preds_svc_train = svc.predict(X_train)
"""

'\nY_preds_svc_train = svc.predict(X_train)\n'

In [43]:
#Model 3
"""
mlp = MLPClassifier(hidden_layer_sizes=(4,8,16,32,64,32,16,8,4),random_state = 42,learning_rate = "adaptive",max_iter = 40000)
mlp.fit(X_train,Y_train)
Y_preds_mlp = mlp.predict(X_test)
clr_mlp = classification_report(Y_test,Y_preds_mlp)
print(clr_mlp)
"""

'\nmlp = MLPClassifier(hidden_layer_sizes=(4,8,16,32,64,32,16,8,4),random_state = 42,learning_rate = "adaptive",max_iter = 40000)\nmlp.fit(X_train,Y_train)\nY_preds_mlp = mlp.predict(X_test)\nclr_mlp = classification_report(Y_test,Y_preds_mlp)\nprint(clr_mlp)\n'

In [44]:
"""
Y_preds_mlp_train = mlp.predict(X_train)
"""

'\nY_preds_mlp_train = mlp.predict(X_train)\n'

In [45]:
from sklearn.ensemble import RandomForestClassifier
"""
rfc = RandomForestClassifier(ccp_alpha = 1e-4, random_state = 42)
rfc.fit(X_train,Y_train)
Y_preds_rfc = rfc.predict(X_test)
clr_rfc = classification_report(Y_test,Y_preds_rfc)
print(clr_rfc)
"""

'\nrfc = RandomForestClassifier(ccp_alpha = 1e-4, random_state = 42)\nrfc.fit(X_train,Y_train)\nY_preds_rfc = rfc.predict(X_test)\nclr_rfc = classification_report(Y_test,Y_preds_rfc)\nprint(clr_rfc)\n'

In [46]:
"""
Y_preds_rfc_train = rfc.predict(X_train)
"""

'\nY_preds_rfc_train = rfc.predict(X_train)\n'

In [47]:
from sklearn.ensemble import ExtraTreesClassifier
"""
etc = ExtraTreesClassifier(ccp_alpha = 1e-4,random_state = 22)
etc.fit(X_train,Y_train)
Y_preds_etc = etc.predict(X_test)
clr_etc = classification_report(Y_test,Y_preds_etc)
print(clr_etc)
"""

'\netc = ExtraTreesClassifier(ccp_alpha = 1e-4,random_state = 22)\netc.fit(X_train,Y_train)\nY_preds_etc = etc.predict(X_test)\nclr_etc = classification_report(Y_test,Y_preds_etc)\nprint(clr_etc)\n'

In [48]:
"""
Y_preds_etc_train = etc.predict(X_train)
"""

'\nY_preds_etc_train = etc.predict(X_train)\n'

In [49]:
from sklearn.tree import DecisionTreeClassifier
"""
dtc = DecisionTreeClassifier(ccp_alpha = 0,random_state = 100)
dtc.fit(X_train,Y_train)
Y_preds_dtc = dtc.predict(X_test)
clr_dtc = classification_report(Y_test,Y_preds_dtc)
print(clr_dtc)
"""

'\ndtc = DecisionTreeClassifier(ccp_alpha = 0,random_state = 100)\ndtc.fit(X_train,Y_train)\nY_preds_dtc = dtc.predict(X_test)\nclr_dtc = classification_report(Y_test,Y_preds_dtc)\nprint(clr_dtc)\n'

In [50]:
"""
Y_preds_dtc_train = dtc.predict(X_train)
"""

'\nY_preds_dtc_train = dtc.predict(X_train)\n'

In [51]:
from sklearn.ensemble import AdaBoostClassifier
"""
abc = AdaBoostClassifier(learning_rate = 1e-6,random_state = 42)
abc.fit(X_train,Y_train)
Y_preds_abc = abc.predict(X_test)
clr_abc = classification_report(Y_test,Y_preds_abc)
print(clr_abc)
"""

'\nabc = AdaBoostClassifier(learning_rate = 1e-6,random_state = 42)\nabc.fit(X_train,Y_train)\nY_preds_abc = abc.predict(X_test)\nclr_abc = classification_report(Y_test,Y_preds_abc)\nprint(clr_abc)\n'

In [52]:
"""
Y_preds_abc_train = abc.predict(X_train)
"""

'\nY_preds_abc_train = abc.predict(X_train)\n'

In [53]:
from sklearn.ensemble import BaggingClassifier
"""
bc = BaggingClassifier(random_state = 42)
bc.fit(X_train,Y_train)
Y_preds_bc = bc.predict(X_test)
clr_bc = classification_report(Y_test,Y_preds_bc)
print(clr_bc)
"""

'\nbc = BaggingClassifier(random_state = 42)\nbc.fit(X_train,Y_train)\nY_preds_bc = bc.predict(X_test)\nclr_bc = classification_report(Y_test,Y_preds_bc)\nprint(clr_bc)\n'

In [54]:
"""
Y_preds_bc_train = bc.predict(X_train)
"""

'\nY_preds_bc_train = bc.predict(X_train)\n'

In [55]:
from sklearn.ensemble import GradientBoostingClassifier
"""
gbc = GradientBoostingClassifier(ccp_alpha = 1e-6,random_state = 42)
gbc.fit(X_train,Y_train)
Y_preds_gbc = gbc.predict(X_test)
clr_gbc = classification_report(Y_test,Y_preds_gbc)
print(clr_gbc)
"""

'\ngbc = GradientBoostingClassifier(ccp_alpha = 1e-6,random_state = 42)\ngbc.fit(X_train,Y_train)\nY_preds_gbc = gbc.predict(X_test)\nclr_gbc = classification_report(Y_test,Y_preds_gbc)\nprint(clr_gbc)\n'

In [56]:
"""
Y_preds_gbc_train = gbc.predict(X_train)
"""

'\nY_preds_gbc_train = gbc.predict(X_train)\n'

In [57]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgbc = HistGradientBoostingClassifier(learning_rate= 0.1, max_iter= 100, max_leaf_nodes= 25, random_state= 10,l2_regularization = 0.01,validation_fraction = 0.1,min_samples_leaf = 20)
hgbc.fit(X,Y)
Y_preds_hgbc = hgbc.predict(X_test)
clr_hgbc = classification_report(Y_test,Y_preds_hgbc)
print(clr_hgbc)

              precision    recall  f1-score   support

           0       0.78      0.56      0.65      1105
           1       0.71      0.87      0.78      1371

    accuracy                           0.73      2476
   macro avg       0.75      0.72      0.72      2476
weighted avg       0.74      0.73      0.73      2476



In [58]:
#a = HistGradientBoostingClassifier()

In [59]:
from sklearn.model_selection import GridSearchCV
"""
param_grid = {"learning_rate":[0.1,0.05],"max_iter":[100,200], "max_leaf_nodes" : [20,31,42],"random_state":[10,42]}
skf = StratifiedKFold(shuffle = True,random_state = 0)
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(test_size = 0.2, random_state = 42)
search = GridSearchCV(a,param_grid,cv =sss)
search.fit(X,Y)
"""

'\nparam_grid = {"learning_rate":[0.1,0.05],"max_iter":[100,200], "max_leaf_nodes" : [20,31,42],"random_state":[10,42]}\nskf = StratifiedKFold(shuffle = True,random_state = 0)\nfrom sklearn.model_selection import StratifiedShuffleSplit\nsss = StratifiedShuffleSplit(test_size = 0.2, random_state = 42)\nsearch = GridSearchCV(a,param_grid,cv =sss)\nsearch.fit(X,Y)\n'

In [60]:
"""
search_preds = search.predict(X_test)
clr_search = classification_report(Y_test,search_preds)
print(clr_search)
"""

'\nsearch_preds = search.predict(X_test)\nclr_search = classification_report(Y_test,search_preds)\nprint(clr_search)\n'

In [61]:
"""
print(search.best_params_)
"""

'\nprint(search.best_params_)\n'

In [62]:
"""
Y_preds_hgbc_train = hgbc.predict(X_train)
"""

'\nY_preds_hgbc_train = hgbc.predict(X_train)\n'

In [63]:
#Create a new dataframe that uses the predictions of all these classifiers and then finally applies an MLP to hopefully get better results.
"""
results = pd.DataFrame()
results["Logistic"] = Y_preds
results["SVC"] = Y_preds_svc
results["MLP"] = Y_preds_mlp
results["RFC"] = Y_preds_rfc
results["ETC"] = Y_preds_etc
results["DTC"] = Y_preds_dtc
results["ABC"] = Y_preds_abc
results["BC"] = Y_preds_bc
results["GBC"] = Y_preds_gbc
results["HGBC"] = Y_preds_hgbc
"""

'\nresults = pd.DataFrame()\nresults["Logistic"] = Y_preds\nresults["SVC"] = Y_preds_svc\nresults["MLP"] = Y_preds_mlp\nresults["RFC"] = Y_preds_rfc\nresults["ETC"] = Y_preds_etc\nresults["DTC"] = Y_preds_dtc\nresults["ABC"] = Y_preds_abc\nresults["BC"] = Y_preds_bc\nresults["GBC"] = Y_preds_gbc\nresults["HGBC"] = Y_preds_hgbc\n'

In [64]:
"""
train_results = pd.DataFrame()
train_results["Logistic"] = Y_preds_train
train_results["SVC"] = Y_preds_svc_train
train_results["MLP"] = Y_preds_mlp_train
train_results["RFC"] = Y_preds_rfc_train
train_results["ETC"] = Y_preds_etc_train
train_results["DTC"] = Y_preds_dtc_train
train_results["ABC"] = Y_preds_abc_train
train_results["BC"] = Y_preds_bc_train
train_results["GBC"] = Y_preds_gbc_train
train_results["HGBC"] = Y_preds_hgbc_train
"""

'\ntrain_results = pd.DataFrame()\ntrain_results["Logistic"] = Y_preds_train\ntrain_results["SVC"] = Y_preds_svc_train\ntrain_results["MLP"] = Y_preds_mlp_train\ntrain_results["RFC"] = Y_preds_rfc_train\ntrain_results["ETC"] = Y_preds_etc_train\ntrain_results["DTC"] = Y_preds_dtc_train\ntrain_results["ABC"] = Y_preds_abc_train\ntrain_results["BC"] = Y_preds_bc_train\ntrain_results["GBC"] = Y_preds_gbc_train\ntrain_results["HGBC"] = Y_preds_hgbc_train\n'

In [65]:
"""
X_train_final = train_results
Y_train_final = Y_train.copy()
X_test_final = results
Y_test_final = Y_test.copy()
mlp_final = MLPClassifier(hidden_layer_sizes=(4,8,16,32,64,32,16,8,4),random_state = 42,learning_rate = "invscaling",max_iter = 40000)
mlp_final.fit(X_train_final,Y_train_final)
Y_preds_final = mlp_final.predict(X_test_final)
clr_final = classification_report(Y_test_final,Y_preds_final)
print(clr_final)
"""

'\nX_train_final = train_results\nY_train_final = Y_train.copy()\nX_test_final = results\nY_test_final = Y_test.copy()\nmlp_final = MLPClassifier(hidden_layer_sizes=(4,8,16,32,64,32,16,8,4),random_state = 42,learning_rate = "invscaling",max_iter = 40000)\nmlp_final.fit(X_train_final,Y_train_final)\nY_preds_final = mlp_final.predict(X_test_final)\nclr_final = classification_report(Y_test_final,Y_preds_final)\nprint(clr_final)\n'

## For submission:

In [66]:
test_data = pd.read_csv(r"../input/marketing-strategy-personalised-offer/test_data.csv")

In [67]:
len(test_data)

5305

In [68]:
test_data["no_visited_Cold drinks"].replace(np.nan,"less1", inplace = True)
test_data["Restaur_spend_less_than20"].replace(np.nan,"1~3", inplace = True)
test_data["no_visited_bars"].replace(np.nan,"never", inplace = True)
test_data["car"].replace(np.nan,"Car that is too old to install Onstar :D", inplace = True)
test_data["no_Take-aways"].replace(np.nan,"1~3",inplace = True)
test_data["Restaur_spend_greater_than20"].replace(np.nan,"less1",inplace = True)

In [69]:
test_data.isna().sum()

offer expiration                        0
income_range                            0
no_visited_Cold drinks                  0
travelled_more_than_15mins_for_offer    0
Restaur_spend_less_than20               0
Marital Status                          0
restaurant type                         0
age                                     0
Prefer western over chinese             0
travelled_more_than_25mins_for_offer    0
travelled_more_than_5mins_for_offer     0
no_visited_bars                         0
gender                                  0
car                                     0
restuarant_same_direction_house         0
Cooks regularly                         0
Customer type                           0
Qualification                           0
is foodie                               0
no_Take-aways                           0
Job/Job Industry                        0
restuarant_opposite_direction_house     0
has Children                            0
visit restaurant with rating (avg)

In [70]:
test_data["temperature"] = ss.transform(test_data["temperature"].values.reshape(-1,1))
test_data["Travel Time"] = ss2.transform(test_data["Travel Time"].values.reshape(-1,1))
#test_data["visit restaurant with rating (avg)"] = ss3.transform(test_data["visit restaurant with rating (avg)"].values.reshape(-1,1))
test_data_numerical = pd.get_dummies(test_data)
X_test_new = test_data_numerical

In [71]:
"""
test_results = pd.DataFrame()
test_results["Logistic"] = lr.predict(X_test_new)
test_results["SVC"] = svc.predict(X_test_new)
test_results["MLP"] = mlp.predict(X_test_new)
test_results["RFC"] = rfc.predict(X_test_new)
test_results["ETC"] = etc.predict(X_test_new)
test_results["DTC"] = dtc.predict(X_test_new)
test_results["ABC"] = abc.predict(X_test_new)
test_results["BC"] = bc.predict(X_test_new)
test_results["GBC"] = gbc.predict(X_test_new)
test_results["HGBC"] = hgbc.predict(X_test_new)
"""

'\ntest_results = pd.DataFrame()\ntest_results["Logistic"] = lr.predict(X_test_new)\ntest_results["SVC"] = svc.predict(X_test_new)\ntest_results["MLP"] = mlp.predict(X_test_new)\ntest_results["RFC"] = rfc.predict(X_test_new)\ntest_results["ETC"] = etc.predict(X_test_new)\ntest_results["DTC"] = dtc.predict(X_test_new)\ntest_results["ABC"] = abc.predict(X_test_new)\ntest_results["BC"] = bc.predict(X_test_new)\ntest_results["GBC"] = gbc.predict(X_test_new)\ntest_results["HGBC"] = hgbc.predict(X_test_new)\n'

In [72]:
"""
testing_final = test_results.copy()
Y_preds_test = mlp_final.predict(testing_final)
"""

'\ntesting_final = test_results.copy()\nY_preds_test = mlp_final.predict(testing_final)\n'

In [73]:
"""
sample = pd.read_csv(r"../input/marketing-strategy-personalised-offer/sample.csv")
sample.columns
"""

'\nsample = pd.read_csv(r"../input/marketing-strategy-personalised-offer/sample.csv")\nsample.columns\n'

In [74]:
#sample

In [75]:
outs = pd.DataFrame()
only_hgbc = hgbc.predict(X_test_new)
#only_mlp = mlp.predict(X_test_new)
outs["id"] = [i for i in range(len(only_hgbc))]
outs["Offer Accepted"] = ["Yes" if item == 1 else "No" for item in only_hgbc]

In [76]:
print("Finally Submitting to competition!")

Finally Submitting to competition!


In [77]:
#Submitting the file
outs.to_csv(r"submission.csv", index = False)